## GR Nuclear Cytoplasmic Ratio Calculation

**GR Processing Outline**

1. Use FISH Analysis to segment nuclei to get nuc_masks for each timepoint 

2. Load in nuc_masks for each image 
2. Dialate nuc_mask by X pixels to get dialated_nuc_masks
3. dialated_nuc_masks - nuc_masks = cyto_ROI
4. Load in GR ICC channel from images 

    a. Use sharpness finder to find best_z

    b. GR_image = max projection of ± 2 z's around best_z

5. Multiply nuc_masks by GR_image to get nuc_GR
6. Multiply cyto_ROI by Gr_image to get cyto_GR
7. Pair nuc_GR and cyto_GR for each cell 
8. Calculate nuc_cyto_ratio = nuc_GR / cyto_GR
9. Save nuc_GR, cyto_GR, nuc_Cyto_ratio for all cells (normalize intensity??)


## Import Packages


In [ ]:
import os
import cv2
import numpy as np
import skimage.filters as skif
from skimage.measure import label
from scipy.signal import convolve2d
import tifffile

## Defining Paths

In [ ]:
# Set the directory containing the TIFF files
images_dir = "Users/ericron/Desktop/ICC_Processing/images/GR_ICC_0min"
mask_dir = "/Users/ericron/Desktop/ICC_Processing/segmented_images/masks_GR_ICC_10nM_Dex_0min_050223"
# Set the output directory for nuclear intensity
nuc_output_dir = "/Users/ericron/Desktop/ICC_Processing/Nuc_Cyto_intensities/nuc_intensity_0min"
# Set the output directory for cytoplasmic intensity
cyto_output_dir = "Users/ericron/Desktop/ICC_Processing/Nuc_Cyto_intensities/cyto_intensity_0min"

# Image directory for NAS Files
# 'CO' is GR ICC channel
# 'C1' is Nuc/DAPI channel

# mandatory substring = 'nuc_100__cyto_200__psfz_350__psfyx_160'

# GR_ICC_R1=(\
# 'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_0min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_10min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_20min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_30min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_40min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_50min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_60min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_75min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_90min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_120min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_150min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_180min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_10nM_0min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_10nM_10min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_10nM_20min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_30min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_40min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_50min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_60min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_75min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_90min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_120min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_150min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_180min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_0min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_10min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_20min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_30min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_40min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_50min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_60min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_75min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_90min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_120min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_150min_050823' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_180min_050823' ,
# )

# Nuclear Mask directory for NAS Files


# GR_ICC_R1=(\
# 'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_0min_050823/masks_GR_ICC_3hr_R1_1nM_0min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_10min_050823/masks_GR_ICC_3hr_R1_1nM_10min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_20min_050823/masks_GR_ICC_3hr_R1_1nM_20min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_30min_050823/masks_GR_ICC_3hr_R1_1nM_30min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_40min_050823/masks_GR_ICC_3hr_R1_1nM_40min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230511/GR_ICC_3hr_R1_1nM_50min_050823/masks_GR_ICC_3hr_R1_1nM_50min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_60min_050823/masks_GR_ICC_3hr_R1_1nM_60min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_75min_050823/masks_GR_ICC_3hr_R1_1nM_75min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_90min_050823/masks_GR_ICC_3hr_R1_1nM_90min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_120min_050823/masks_GR_ICC_3hr_R1_1nM_120min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_150min_050823/masks_GR_ICC_3hr_R1_1nM_150min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_1nM_180min_050823/masks_GR_ICC_3hr_R1_1nM_180min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_10nM_0min_050823/masks_GR_ICC_3hr_R1_10nM_0min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_10nM_10min_050823/masks_GR_ICC_3hr_R1_10nM_10min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230516/GR_ICC_3hr_R1_10nM_20min_050823/masks_GR_ICC_3hr_R1_10nM_20min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_30min_050823/masks_GR_ICC_3hr_R1_10nM_30min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_40min_050823/masks_GR_ICC_3hr_R1_10nM_40min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_50min_050823/masks_GR_ICC_3hr_R1_10nM_50min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_60min_050823/masks_GR_ICC_3hr_R1_10nM_60min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_75min_050823/masks_GR_ICC_3hr_R1_10nM_75min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_90min_050823/masks_GR_ICC_3hr_R1_10nM_90min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_120min_050823/masks_GR_ICC_3hr_R1_10nM_120min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_150min_050823/masks_GR_ICC_3hr_R1_10nM_150min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_10nM_180min_050823/masks_GR_ICC_3hr_R1_10nM_180min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_0min_050823/masks_GR_ICC_3hr_R1_100nM_0min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_10min_050823/masks_GR_ICC_3hr_R1_100nM_10min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_20min_050823/masks_GR_ICC_3hr_R1_100nM_20min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_30min_050823/masks_GR_ICC_3hr_R1_100nM_30min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230522/GR_ICC_3hr_R1_100nM_40min_050823/masks_GR_ICC_3hr_R1_100nM_40min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_50min_050823/masks_GR_ICC_3hr_R1_100nM_50min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_60min_050823/masks_GR_ICC_3hr_R1_100nM_60min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_75min_050823/masks_GR_ICC_3hr_R1_100nM_75min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_90min_050823/masks_GR_ICC_3hr_R1_100nM_90min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_120min_050823/masks_GR_ICC_3hr_R1_100nM_120min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_150min_050823/masks_GR_ICC_3hr_R1_100nM_150min_050823___nuc_100__cyto_200.zip' ,
# 'smFISH_images/Eric_smFISH_images/20230530/GR_ICC_3hr_R1_100nM_180min_050823/masks_GR_ICC_3hr_R1_100nM_180min_050823___nuc_100__cyto_200.zip' ,
# )


## Z-Stack to Single-Z

In [ ]:
# # Create max projections of GR images
# for filename in os.listdir(images_dir):
#     if filename.endswith(".tif") and "C0" in images_dir:
#         # Load the TIFF file
#         filepath = os.path.join(images_dir, filename)
#         img = tifffile.imread(filepath)

#         # Combine all z plane images using maximum projection
#         img_max = np.max(img, axis=0)

#         # Convert the image to 16-bit
#         img_16bit = cv2.normalize(img_max, None, 0, 65535, cv2.NORM_MINMAX, cv2.CV_16U)

In [ ]:
#Sharpness finder for best z using GR Image

img = "/Users/ericron/Desktop/ICC_Processing/test/ROI001_XY1685471975_Z00_T0_C0.tif"
imag = tifffile.imread(img)
kernel = np.array([[0,-1,0],[-1,4,-1],[0,-1,0]])
# print(imag.shape)
# print(imag)
# print(kernel)

# print(sharpnessValue)
#/np.sum(np.sum(imag[i,:,:]))
sharpnessValue=[]
for i in range(imag.shape[0]):
    sharpnessValue.append(np.sum(np.sum(np.abs(convolve2d(imag[i,:,:],kernel)))))

print(sharpnessValue)
max_index = np.argmax(sharpnessValue)


## Nuclear to Cytoplasmic Calculation

In [ ]:
import numpy as np
from scipy.ndimage import binary_dilation, label
import tifffile
import matplotlib.pyplot as plt
from PIL import Image
import skimage as sk
from skimage.io import imread 
from skimage import measure
import pandas as pd

# mask_test = '/Users/ericron/Desktop/ICC_Processing/segmented_images/masks_GR_ICC_10nM_Dex_0min_050223/masks_nuclei_ROI001_XY1683126754_Z00_T0_merged.tif'
mask_test = '/Users/ericron/Desktop/ICC_Processing/segmented_images/masks_GR_ICC_10nM_Dex_30min_050223/masks_nuclei_ROI001_XY1683128269_Z00_T0_merged.tif'

# Load the mask TIFF image
image_mask = tifffile.imread(mask_test)

# Define the number of pixels to dilate
num_pixels_to_dilate = 4

# Set a threshold value
threshold = 1

# Binarize the grayscale image
image_mask = np.where(image_mask >= threshold, 1, 0)

# Create a structuring element with a size of num_pixels_to_dilate
structuring_element = np.ones((num_pixels_to_dilate, num_pixels_to_dilate), dtype=np.uint16)

# Perform dilation on the binary image with the specified number of iterations
dilated_image_mask = binary_dilation(image_mask, structure=structuring_element, iterations=num_pixels_to_dilate)

# Display the nuclei masks
plt.subplot(1, 2, 1)
plt.imshow(image_mask, cmap='viridis')
plt.title('Nuclei Masks')

# Display the dilated masks
plt.subplot(1, 2, 2)
plt.imshow(dilated_image_mask, cmap='viridis')
plt.title('Dilated Masks')

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

cyto_mask = np.subtract(dilated_image_mask, image_mask)

# Display the cytoplasmic mask
plt.imshow(cyto_mask, cmap='viridis')
plt.axis('off')
plt.show()
plt.title('Cytoplasmic Masks')

# gr_image = tifffile.imread('/Users/ericron/Desktop/ICC_Processing/images/MAX_ROI001_XY1683126754_Z00_T0_C0.tif')
gr_image = tifffile.imread('/Users/ericron/Desktop/ICC_Processing/images/MAX_ROI001_XY1683128269_Z00_T0_C0.tif')

# Display the GR image
plt.imshow(gr_image, cmap='gray')
plt.axis('off')
plt.show()
plt.title('0min GR ICC')

nuclei = np.multiply(image_mask,gr_image)
cytoplasm = np.multiply(cyto_mask,gr_image)

# Display the nuclei image
plt.imshow(nuclei, cmap='gray')
plt.axis('off')
plt.title('Nuclei')
plt.colorbar()
plt.show()

# Display the Cytoplasm image
plt.imshow(cytoplasm, cmap='gray')
plt.axis('off')
plt.title('Cytoplasm')
plt.colorbar()
plt.show()


# Perform connected component analysis on grayscale images
labeled_nuclei, num_nuclei = label(nuclei)
labeled_cytoplasm, num_cytoplasm = label(cytoplasm)

print("Number of nuclei:", num_nuclei)
print("Number of cytoplasm:", num_cytoplasm)

# Display the labeled nuclei image
plt.imshow(labeled_nuclei)
plt.axis('off')
plt.title('Labeled Nuclei')
plt.colorbar()
plt.show()

# Display the labeled cytoplasm image
plt.imshow(labeled_cytoplasm)
plt.axis('off')
plt.title('Labeled Cytoplasm')
plt.colorbar()
plt.show()

# Calculate average nuclear intensities
avg_nuc_intensity_list = []
for label_idx in range(1, num_nuclei + 1):
    avg_intensity = np.mean(nuclei[labeled_nuclei == label_idx])
    avg_nuc_intensity_list.append(avg_intensity)

# Calculate average cytoplasmic intensities
avg_cyto_intensity_list = []
for label_idx in range(1, num_cytoplasm + 1):
    avg_intensity = np.mean(cytoplasm[labeled_cytoplasm == label_idx])
    avg_cyto_intensity_list.append(avg_intensity)

# Nuclear to Cytoplasmic Ratio
NCR = np.array(avg_nuc_intensity_list) / np.array(avg_cyto_intensity_list)
NCR_list = NCR.tolist()

# Generate cell numbers
cell_number_list = []
for label_idx in range(1, num_nuclei + 1):
    cell_number_list.append(label_idx)

# Create a DataFrame
data = {
    'Cell Number': cell_number_list,
    'Average Nuclear Intensity': avg_nuc_intensity_list,
    'Average Cytoplasmic Intensity': avg_cyto_intensity_list,
    'Nuclear to Cytoplasmic Ratio': NCR_list
}

df = pd.DataFrame(data)
